In [74]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
import pickle
#from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.models import load_model
import re
import random
import sys
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
import io
from pickle import dump
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [75]:
from nltk.corpus import gutenberg

tokens_raw1 = gutenberg.raw('austen-emma.txt').lower()
n1 = len(tokens_raw1)
tokens_train1  = tokens_raw1[0:int(0.8*n1)];
tokens_test1 = tokens_raw1[int(0.8*n1):n1]; 

tokens_raw2 = gutenberg.raw('austen-sense.txt').lower()
n2 = len(tokens_raw2)
tokens_train2  = tokens_raw2[0:int(0.8*n2)];
tokens_test2 = tokens_raw2[int(0.8*n2):n2]; 

tokens_raw = tokens_train1 + tokens_train2
tokens_test = tokens_test1 + tokens_test2


In [76]:
char_set = sorted(list(set(tokens_raw)))

print(char_set)
print(len(char_set))
char_index_dict={}
index_char_dict={}
for i in range(len(char_set)):
    char_index_dict[char_set[i]]=i
    index_char_dict[i]=char_set[i]

tokens_raw=list(tokens_raw)

char_tr=[]
for t in tokens_raw:
    if(t!='\n'):
        char_tr.append(t)
    else:
        char_tr.append(" ")
        


length = 41
sequences =[]
for i in range(length, len(char_tr),2):
    seq = char_tr[i-length:i]
    sequences.append(seq)
print(sequences[1])
lines = sequences;
print('Total Sequences: %d' % len(sequences))
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)

[u'\n', u' ', u'!', u'"', u'&', u"'", u'(', u')', u'*', u',', u'-', u'.', u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u':', u';', u'?', u'[', u']', u'_', u'`', u'a', u'b', u'c', u'd', u'e', u'f', u'g', u'h', u'i', u'j', u'k', u'l', u'm', u'n', u'o', u'p', u'q', u'r', u's', u't', u'u', u'v', u'w', u'x', u'y', u'z']
55
[u'm', u'm', u'a', u' ', u'b', u'y', u' ', u'j', u'a', u'n', u'e', u' ', u'a', u'u', u's', u't', u'e', u'n', u' ', u'1', u'8', u'1', u'6', u']', ' ', ' ', u'v', u'o', u'l', u'u', u'm', u'e', u' ', u'i', ' ', ' ', u'c', u'h', u'a', u'p', u't']
Total Sequences: 624016


In [5]:
#creating tokens of 40 charactars to predict 41st charactar
X=[]
y=[]
for i in range(len(sequences)):
    X.append(sequences[i][:-1])
    y.append(sequences[i][-1])
X=np.array(X)
y=np.array(y).reshape(-1,1)
print(X.shape)
print(y.shape)

(624016, 40)
(624016, 1)


In [77]:
# define model
vocab_size = len(tokenizer.word_index)+1
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=40))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

                





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            2750      
_________________________________________________________________
lstm_3 (LSTM)                (None, 40, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 55)                5555      
Total params: 159,205
Trainable params: 159,205
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, batch_size=200, epochs=25)

Epoch 1/25
624016/624016 [==============================] - 787s 1ms/step - loss: 2.1955 - acc: 0.3643
Epoch 2/25
624016/624016 [==============================] - 782s 1ms/step - loss: 1.7100 - acc: 0.4879
Epoch 3/25
624016/624016 [==============================] - 782s 1ms/step - loss: 1.5484 - acc: 0.5316
Epoch 4/25
624016/624016 [==============================] - 783s 1ms/step - loss: 1.4582 - acc: 0.5567
Epoch 5/25
624016/624016 [==============================] - 783s 1ms/step - loss: 1.4010 - acc: 0.5715
Epoch 6/25
624016/624016 [==============================] - 784s 1ms/step - loss: 1.3601 - acc: 0.5826
Epoch 7/25
624016/624016 [==============================] - 784s 1ms/step - loss: 1.3290 - acc: 0.5901
Epoch 8/25
624016/624016 [==============================] - 785s 1ms/step - loss: 1.3039 - acc: 0.5968
Epoch 9/25
624016/624016 [==============================] - 785s 1ms/step - loss: 1.2841 - acc: 0.6020
Epoch 10/25
624016/624016 [==============================] - 786s 1ms/ste

In [37]:

# save the model to file
model.save('model_a03_cl.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_a03_cl.pkl', 'wb'))

In [78]:
from random import randint
from pickle import load
from keras.models import load_model
# load the model
model = load_model('model_a03_cl.h5')
# load the tokenizer
tokenizer = load(open('tokenizer_a03_cl.pkl', 'rb'))

In [202]:
# In[]
from keras.preprocessing.sequence import pad_sequences


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        #yhat1 = model.predict(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text.append(out_word)
        result.append(out_word)
    return ''.join(result)

In [201]:
#seed_text = lines[randint(0,len(lines))]
in_text = seed_text
encoded = tokenizer.texts_to_sequences([in_text])[0]
print(encoded)
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
yhat = model.predict_classes(encoded, verbose=0)
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break
                
in_text.append(out_word)
print(in_text)

[2, 28, 1, 5, 6, 11, 1, 3, 9, 2, 1, 10, 2, 14, 2, 14, 21, 10, 5, 6, 17, 2, 1, 4, 18, 1, 5, 12, 12, 1, 9, 2, 10, 1, 18, 4, 10, 14, 2, 10, 1]
[u'e', u';', u' ', u'a', u'n', u'd', u' ', u't', u'h', u'e', u' ', u'r', u'e', u'm', u'e', u'm', u'b', u'r', u'a', u'n', u'c', u'e', ' ', u'o', u'f', u' ', u'a', u'l', u'l', u' ', u'h', u'e', u'r', u' ', u'f', u'o', u'r', u'm', u'e', u'r', u' ', u't']


In [203]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(''.join(seed_text) + '')
generated = generate_seq(model, tokenizer, length-1, seed_text, 80)
print(generated)

all the tediousness of the many years of 
the same old from the conscious and seeing the door, and they were a serious of 


In [214]:
#perplexity
seq_length = length-1
test_tokens = tokens_test[1:10000]

in_text = [test_tokens[0]]
logp = 0
c=0
for word in test_tokens:
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    #print encoded
    yhat1 = model.predict(encoded,verbose=0)[0]
    next_word = word
    if next_word in tokenizer.word_index:
        p = yhat1[tokenizer.word_index[word]]
        if  p !=0:
            c+=1
            logp = logp + np.log(p)

    else:
        p = 1
        
        c+=1
        logp = logp + np.log(p)
    
    in_text.append(word)
    #print(in_text)
    
pplxty = np.exp(-1*logp/c)
print c

9999


In [211]:
pplxty

4.569143307682693